In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

# Assuming you have your data in a DataFrame with features in 'X' and target in 'y'
# Replace this with your actual data loading process
data = pd.read_csv('/media/masharifin/D Drive/ComprehensiveExam/Datasets/testsmote.csv')
data['Class'] = data['Class'].replace({'Benign': 0, 'Malware': 1})
X = data.drop(columns=['Class'])
y = data['Class']


train_size = 0.6  # 70% for training, 30% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)


# Create and train the XGBoost classifier (replace with your own model training code)
xgboost_model = xgb.XGBClassifier()
xgboost_model.fit(X_train, y_train)


def fgsm_attack(model, sample, epsilon):
    # Convert the sample to a numpy array if it's a pandas Series
    if isinstance(sample, pd.Series):
        sample = sample.values
    
    # Convert the sample to a 2D numpy array if it's 1D
    if len(sample.shape) == 1:
        sample = sample.reshape(1, -1)

    # Get the original prediction of the model for the given sample
    original_pred = model.predict(sample)

    # Calculate the perturbation for each feature separately
    perturbation = np.zeros_like(sample)
    for feature_index in range(sample.shape[1]):
        # Create a copy of the sample to avoid modifying the original array
        sample_copy = sample.copy()
        # Add perturbation to the current feature
        sample_copy[:, feature_index] += epsilon
        # Calculate the new prediction for the perturbed sample
        perturbed_pred = model.predict(sample_copy)
        # Calculate the perturbation for the current feature
        perturbation[:, feature_index] = epsilon * np.sign(perturbed_pred - original_pred)

    # Generate the adversarial example by adding the perturbation to the original sample
    adversarial_sample = sample + perturbation
    
    return adversarial_sample


# Create an empty array to store the adversarial samples
adversarial_samples = []

# Iterate over each sample in the dataset
for sample_index in range(len(X_train)):
    # Select the sample and reshape it to (1, num_features)
    sample = X_train.iloc[sample_index].values.reshape(1, -1)
    true_label = y_train.iloc[sample_index]

    # Generate the adversarial example using the FGSM attack function
    epsilon = 3 # Set the epsilon value for the attack
    adversarial_sample = fgsm_attack(xgboost_model, sample, epsilon)

    # Store the adversarial sample in the array
    adversarial_samples.append(adversarial_sample)

# Convert the list of adversarial samples into a NumPy array
adversarial_samples = np.concatenate(adversarial_samples, axis=0)

# Now adversarial_samples contains the adversarial examples for the entire dataset

adversarial_sample = fgsm_attack(xgboost_model, sample, epsilon)


# Get the model's prediction on the original sample and the adversarial sample
original_prediction = xgboost_model.predict(sample)[0]
adversarial_prediction = xgboost_model.predict(adversarial_sample)[0]

# Check if the model's prediction on the adversarial sample is different from the true label
if adversarial_prediction != true_label:
    print("Attack Successful! Model misclassified the adversarial example.")
else:
    print("Attack Unsuccessful! Model correctly classified the adversarial example.")


Attack Unsuccessful! Model correctly classified the adversarial example.
